# ZERO-INFLATED NEGATIVE BINOMIAL REGRESSION | R DATA ANALYSIS EXAMPLES

*A reproduction of UCLA IDRE Stat's https://stats.idre.ucla.edu/r/dae/zinb/*

Zero-inflated negative binomial regression is for modeling count variables with excessive zeros and it is usually for overdispersed count outcome variables. Furthermore, theory suggests that the excess zeros are generated by a separate process from the count values and that the excess zeros can be modeled independently.

This page uses the following packages. Make sure that you can load them before trying to run the examples on this page. If you do not have a package installed, run: `install.packages("packagename")`, or if you see the version is out of date, run: `update.packages()`.

In [ ]:
# install.packages("ggplot2")
# install.packages("pscl")
# install.packages("MASS")
# install.packages("boot")
require(ggplot2)
require(pscl)
require(MASS)
require(boot)

In [ ]:
# This notebook has been tested with R 3.4.1; ggplot2 3.1.0; pscl 1.5.2; MASS 7.3.50; boot 1.3.20
# getRversion()
# packageVersion('ggplot2')
# packageVersion('pscl')
# packageVersion('MASS')
# packageVersion('boot')

**Please note:** The purpose of this page is to show how to use various data analysis commands. It does not cover all aspects of the research process which researchers are expected to do. In particular, it does not cover data cleaning and checking, verification of assumptions, model diagnostics or potential follow-up analyses.

## Examples of zero-inflated negative binomial regression

Example 1. School administrators study the attendance behavior of high school juniors at two schools. Predictors of the number of days of absence include gender of the student and standardized test scores in math and language arts.

Example 2. The state wildlife biologists want to model how many fish are being caught by fishermen at a state park. Visitors are asked how long they stayed, how many people were in the group, were there children in the group and how many fish were caught. Some visitors do not fish, but there is no data on whether a person fished or not. Some visitors who did fish did not catch any fish so there are excess zeros in the data because of the people that did not fish.

## Description of the Data

Let’s pursue Example 2 from above.

We have data on 250 groups that went to a park. Each group was questioned about how many fish they caught (`count`), how many children were in the group (`child`), how many people were in the group (`persons`), and whether or not they brought a camper to the park (`camper`).

In addition to predicting the number of fish caught, there is interest in predicting the existence of excess zeros, i.e., the probability that a group caught zero fish. We will use the variables `child`, `persons`, and `camper` in our model. Let’s look at the data.

In [ ]:
zinb <- read.csv("https://stats.idre.ucla.edu/stat/data/fish.csv")
zinb <- within(zinb, {
  nofish <- factor(nofish)
  livebait <- factor(livebait)
  camper <- factor(camper)
})

summary(zinb)

In [ ]:
## histogram with x axis in log10 scale
ggplot(zinb, aes(count, fill = camper)) +
  geom_histogram() +
  scale_x_log10() +
  facet_grid(camper ~ ., margins=TRUE, scales="free_y")

## Analysis methods you might consider

Before we show how you can analyze this with a zero-inflated negative binomial analysis, let’s consider some other methods that you might use.

* OLS Regression – You could try to analyze these data using OLS regression. However, count data are highly non-normal and are not well estimated by OLS regression.
* Zero-inflated Poisson Regression – Zero-inflated Poisson regression does better when the data is not overdispersed, i.e. when variance is not much larger than the mean.
* Ordinary Count Models – Poisson or negative binomial models might be more appropriate if there are not excess zeros.

## Zero-inflated negative binomial regression

A zero-inflated model assumes that zero outcome is due to two different processes. For instance, in the example of fishing presented here, the two processes are that a subject has gone fishing vs. not gone fishing. If not gone fishing, the only outcome possible is zero. If gone fishing, it is then a count process. The two parts of the a zero-inflated model are a binary model, usually a logit model to model which of the two processes the zero outcome is associated with and a count model, in this case, a negative binomial model, to model the count process. The expected count is expressed as a combination of the two processes. Taking the example of fishing again:

$$E(n_{fish caught}=k)=P(\textrm{not gone fishing})∗0+P(\textrm{gone fishing})∗E(y=k|\textrm{gone fishing})$$

To understand the zero-inflated negative binomial regression, let’s start with the negative binomial model. There are multiple parameterizations of the negative binomial model, we focus on NB2. The negative binomial probability density function is:

$$PDF(y;p,r)=\frac{(y_i+r–1)!}{y_i!(r−1)!}p^r_i(1–p_i)^{y_i}$$

where (p) is the probability of (r) successes. From this we can derive the likelihood function, which is given by:

$$ L(\mu;y,\alpha)=\prod_{i=1}^{n}exp\left(y_i\ln\left(\frac{\alpha\mu_i}{1+\alpha\mu_i}\right)−\frac{1}{\alpha}\ln(1+\alpha\mu_i)+\ln\Gamma\left(y_i+\frac{1}{\alpha}\right)−\ln\Gamma(y_i+1)–\ln\Gamma\left(\frac{1}{\alpha}\right)\right)$$

here we find the likelihood of the expected value, $\mu$ given the data and $\alpha$ which allows for dispersion. Typically, this would be expressed as a log likelihood, denoted by script L, $\mathcal{L}$:

$$ \mathcal{L}(\mu;y,\alpha)=\sum_{i=1}^{n}\left(y_i\ln\left(\frac{\alpha\mu_i}{1+\alpha\mu_i}\right)−\frac{1}{\alpha}\ln(1+\alpha\mu_i)+\ln\Gamma\left(y_i+\frac{1}{\alpha}\right)−\ln\Gamma(y_i+1)–\ln\Gamma\left(\frac{1}{\alpha}\right)\right)$$

which can be expressed in terms of our model by replacing $\mu_i$ with $exp(x_i^′\beta)$. Turning to the zero-inflated negative binomial model, the expression of the likelihood function depends on whether the observed value is a zero or greater than zero. From the logistic model of $y_i>1$ versus $y=0$:

$$p=\frac{1}{1+e^{−x^′_i\beta}}$$

and

$$1–p=\frac{1}{1+e^{x^′_i\beta}}$$

then

$$
\mathcal{L} = 
\begin{cases}
    \sum_{i=1}^n \left[\ln(p_i)+(1-p_i)\left(\frac{1}{1+\alpha\mu_i}\right)^{\frac{1}{\alpha}}\right] & \text{if } y_i = 0\\
    \sum_{i=1}^n \left[\ln(p_i)+\ln\Gamma\left(\frac{1}{\alpha}+y_i\right)-\ln\Gamma(y_i+1) –ln\Gamma\left(\frac{1}{\alpha}\right)+\left(\frac{1}{\alpha}\right)\ln\left(\frac{1}{1+\alpha\mu_i}\right)+y_i\ln\left(1-\frac{1}{1+\alpha\mu_i}\right) \right] & \text{if } y_i\gt 0
\end{cases}
$$

Finally, note that R does not estimate $\alpha$ but $\theta$, the inverse of $\alpha$.

Now let’s build up our model. We are going to use the variables `child` and `camper` to model the count in the part of negative binomial model and the variable persons in the logit part of the model. We use the `pscl` to run a zero-inflated negative binomial regression. We begin by estimating the model with the variables of interest.

In [ ]:
m1 <- zeroinfl(count ~ child + camper | persons,
  data = zinb, dist = "negbin", EM = TRUE)
summary(m1)

The output looks very much like the output from two OLS regressions in R.

Below the model call, you will find a block of output containing negative binomial regression coefficients for each of the variables along with standard errors, z-scores, and p-values for the coefficients. A second block follows that corresponds to the inflation model. This includes logit coefficients for predicting excess zeros along with their standard errors, z-scores, and p-values.

All of the predictors in both the count and inflation portions of the model are statistically significant. This model fits the data significantly better than the null model, i.e., the intercept-only model. To show that this is the case, we can compare with the current model to a null model without predictors using chi-squared test on the difference of log likelihoods.

In [ ]:
m0 <- update(m1, . ~ 1)

pchisq(2 * (logLik(m1) - logLik(m0)), df = 3, lower.tail=FALSE)

From the output above, we can see that our overall model is statistically significant.

Note that the model output above does not indicate in any way if our zero-inflated model is an improvement over a standard negative binomial regression. We can determine this by running the corresponding standard negative binomial model and then performing a Vuong test of the two models. We use the `MASS` package to run the standard negative binomial regression.

In [ ]:
summary(m2 <- glm.nb(count ~ child + camper, data = zinb))

In [ ]:
vuong(m1, m2)

* The predictors `child` and `camper` in the part of the negative binomial regression model predicting number of fish caught (`count`) are both significant predictors.
* The predictor `person` in the part of the logit model predicting excessive zeros is statistically significant.
* For these data, the expected change in log(`count`) for a one-unit increase in `child` is -1.515255 holding other variables constant.
* A camper (`camper = 1`) has an expected log(`count`) of 0.879051 higher than that of a non-camper (`camper = 0`) holding other variables constant.
* The log odds of being an excessive zero would decrease by 1.67 for every additional person in the group. In other words, the more people in the group the less likely that the zero would be due to not gone fishing. Put plainly, the larger the group the person was in, the more likely that the person went fishing.
* The Vuong test suggests that the zero-inflated negative binomial model is a significant improvement over a standard negative binomial model.

We can get confidence intervals for the parameters and the exponentiated parameters using bootstrapping. For the negative binomial model, these would be incident risk ratios, for the zero inflation model, odds ratios. We use the boot package. First, we get the coefficients from our original model to use as start values for the model to speed up the time it takes to estimate. Then we write a short function that takes data and indices as input and returns the parameters we are interested in. Finally, we pass that to the boot function and do 1200 replicates, using snow to distribute across four cores. Note that you should adjust the number of cores to whatever your machine has. Also, for final results, one may wish to increase the number of replications to help ensure stable results.

In [ ]:
dput(round(coef(m1, "count"), 4))

In [ ]:
dput(round(coef(m1, "zero"), 4))

In [ ]:
f <- function(data, i) {
  require(pscl)
  m <- zeroinfl(count ~ child + camper | persons,
    data = data[i, ], dist = "negbin",
    start = list(count = c(1.3711, -1.5152, 0.879), zero = c(1.6028, -1.6663)))
  as.vector(t(do.call(rbind, coef(summary(m)))[, 1:2]))
}

set.seed(10)
(res <- boot(zinb, f, R = 1200, parallel = "snow", ncpus = 4))

The results are alternating parameter estimates and standard errors. That is, the first row has the first parameter estimate from our model. The second has the standard error for the first parameter. The third column contains the bootstrapped standard errors, which are considerably larger than those estimated by `zeroinfl`.

Now we can get the confidence intervals for all the parameters. We start on the original scale with percentile and bias adjusted CIs. We also compare these results with the regular confidence intervals based on the standard errors.

In [ ]:
## basic parameter estimates with percentile and bias adjusted CIs
parms <- t(sapply(c(1, 3, 5, 7, 9), function(i) {
  out <- boot.ci(res, index = c(i, i + 1), type = c("perc", "bca"))
  with(out, c(Est = t0, pLL = percent[4], pUL = percent[5],
    bcaLL = bca[4], bcaLL = bca[5]))
}))

## add row names
row.names(parms) <- names(coef(m1))
## print results
parms

In [ ]:
## compare with normal based approximation
confint(m1)

The bootstrapped confidence intervals are considerably wider than the normal based approximation. The bootstrapped CIs are more consistent with the CIs from Stata when using robust standard errors.

Now we can estimate the incident risk ratio (IRR) for the negative binomial model and odds ratio (OR) for the logistic (zero inflation) model. This is done using almost identical code as before, but passing a transformation function to the h argument of `boot.ci`, in this case, `exp` to exponentiate.

In [ ]:
## exponentiated parameter estimates with percentile and bias adjusted CIs
expparms <- t(sapply(c(1, 3, 5, 7, 9), function(i) {
  out <- boot.ci(res, index = c(i, i + 1), type = c("perc", "bca"), h = exp)
  with(out, c(Est = t0, pLL = percent[4], pUL = percent[5],
    bcaLL = bca[4], bcaLL = bca[5]))
}))

## add row names
row.names(expparms) <- names(coef(m1))
## print results
expparms

To better understand our model, we can compute the expected number of fish caught for different combinations of our predictors. In fact, since we are working with essentially categorical predictors, we can compute the expected values for all combinations using the `expand.grid` function to create all combinations and then the `predict` function to do it. Finally we create a graph.

In [ ]:
newdata1 <- expand.grid(0:3, factor(0:1), 1:4)
colnames(newdata1) <- c("child", "camper", "persons")
newdata1$phat <- predict(m1, newdata1)

ggplot(newdata1, aes(x = child, y = phat, colour = factor(persons))) +
  geom_point() +
  geom_line() +
  facet_wrap(~camper) +
  labs(x = "Number of Children", y = "Predicted Fish Caught")

## Things to consider

Here are some issues that you may want to consider in the course of your research analysis.

* Question about the over-dispersion parameter is in general a tricky one. A large over-dispersion parameter could be due to a miss-specified model or could be due to a real process with over-dispersion. Adding an over-dispersion problem does not necessarily improve a miss-specified model.
* The zero inflated negative binomial model has two parts, a negative binomial count model and the logit model for predicting excess zeros, so you might want to review these Data Analysis Example pages, [Negative Binomial Regression](https://stats.idre.ucla.edu/r/dae/negative-binomial-regression/) and [Logit Regression](https://stats.idre.ucla.edu/r/dae/logit-regression/).
* Since zero inflated negative binomial has both a count model and a logit model, each of the two models should have good predictors. The two models do not necessarily need to use the same predictors.
* Problems of perfect prediction, separation or partial separation can occur in the logistic part of the zero-inflated model.
* Count data often use exposure variable to indicate the number of times the event could have happened. You can incorporate exposure (also called an offset) into your model by using the `offset()` function.
* It is not recommended that zero-inflated negative binomial models be applied to small samples. What constitutes a small sample does not seem to be clearly defined in the literature.
* Pseudo-R-squared values differ from OLS R-squareds, please see [FAQ: What are pseudo R-squareds?](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-what-are-pseudo-r-squareds/) for a discussion on this issue.

## References

* Long, J. S. 1997. *Regression Models for Categorical and Limited Dependent Variables*. Thousand Oaks, CA: Sage Publications. Everitt, B. S. and Hothorn, T. [A Handbook of Statistical Analyses Using R](http://cran.r-project.org/web/packages/HSAUR/vignettes/preface.pdf)